# AgroGraphNet: End-to-End Prediction Pipeline

This notebook creates an end-to-end prediction pipeline for real-time crop disease prediction.

## Objectives:
1. Load trained models and create inference pipeline
2. Simulate real-time data ingestion
3. Generate predictions for new data
4. Create early warning system
5. Build interactive dashboard
6. Export results for stakeholders

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import torch
import torch.nn as nn
from torch_geometric.data import Data, DataLoader
import folium
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle
import json
import joblib
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from config import *
from model_utils import *
from graph_utils import *
from data_utils import *
from visualization import *

print("Libraries imported successfully!")